# Extended Data Figure 6

![title](../assets/EDFig6.png)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import logging
from pathlib import Path
from itertools import combinations

logging.getLogger("matplotlib.font_manager").disabled = True

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import common

sys.path.append('./prepare_data')

DATA_PATH = Path(
    "../data"
)
EXPORT_PATH = Path(
    "../results/EDFigure6"
)
EXPORT_PATH.mkdir(exist_ok=True, parents=True)

FIGSIZE = {'width': 1.2, 'length': 0.9}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extended Data Figure 6: occupancy of coxa-trochanter and tibia-tarsus foreleg joints

In [2]:
all_positions_intact = common.load_data(DATA_PATH / "EDFig6_intact.pkl")
all_positions_hf = common.load_data(DATA_PATH / "EDFig6_HF.pkl")

limits = {
    "Femur": {
        "x": [0.1, 0.5],
        "y": [-0.4, 0.4],
        "z": [0.6, 0.8],
    },
    "Tibia": {
        "x": [0.4, 1],
        "y": [-0.9, 0.9],
        "z": [0.7, 1.2],
    },
    "Tarsus": {
        "x": [0.5, 1.5],
        "y": [-0.9, 0.9],
        "z": [0.2, 1.6],
    },
}

joint_name = {
    "Femur": "coxa-trochanter joint",
    "Tarsus": "tibia-tarsus joint",
}

for leg_segment in ["Femur", "Tarsus"]:
    data_all = {}
    boundaries = []
    for x, y in combinations(["x", "y", "z"], 2):

        data1, data2 = common.convert_dict_to_array(
            all_positions_intact,
            all_positions_hf,
            [
                f"Pose_RF_{leg_segment}_{x}",
                f"Pose_LF_{leg_segment}_{x}",
            ],
            [
                f"Pose_RF_{leg_segment}_{y}",
                f"Pose_LF_{leg_segment}_{y}",
            ],
        )
        kde1_vals, kde2_vals, x_g, y_g = common.calculate_kde(data1, data2)
        overlap = common.calculate_overlap(kde1_vals, kde2_vals)

        data_all[x + y] = (
            kde1_vals,
            kde2_vals,
            overlap,
            x_g,
            y_g,
            limits[leg_segment][x],
            limits[leg_segment][y],
        )

    with plt.style.context("./config.mplstyle"):

        common.plot_2d_histograms_3x3(
            data_all,
            figsize=(3.1, 3.3),
            titles=[
                f"Intact",
                f"Head-fixed",
                f"Overlap",
                f"2D occupancy histograms of {joint_name[leg_segment]}",
            ],
            export_path=EXPORT_PATH / f"EDFigure6_Intact_HF_{leg_segment}_2d_histograms.png",
        )